In [1]:
import os
from haystack import Pipeline, Document
from haystack.dataclasses import ChatMessage
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore
from haystack_integrations.components.retrievers.mongodb_atlas import MongoDBAtlasEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack.utils import Secret
from dotenv import load_dotenv

load_dotenv()

os.environ["MONGO_CONNECTION_STRING"] = os.getenv("MONGO_CONNECTION_STRING2")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

document_store = MongoDBAtlasDocumentStore(
    database_name="depato_store",
    collection_name="common_info",
    vector_search_index="vector_index",
    full_text_search_index="search_index"
)

conversation_history = []

rag_pipeline = Pipeline()

rag_pipeline.add_component(
    name="text_embedder",
    instance=SentenceTransformersTextEmbedder()
)

rag_pipeline.add_component(
    name="retriever",
    instance=MongoDBAtlasEmbeddingRetriever(document_store=document_store, top_k=5)
)

prompt_template = """
You are a helpful assistant. Use the following conversation history and retrieved documents to answer the user's question. 
If the documents or history do not contain the answer, say that you don't have enough information.

Conversation History:
{% for message in conversation_history %}
    {{ message.role }}: {{ message.content }}
{% endfor %}

Documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}

Question: {{ query }}
Answer:
"""

rag_pipeline.add_component(
    name="prompt_builder",
    instance=PromptBuilder(template=prompt_template)
)

rag_pipeline.add_component(
    name="llm",
    instance=OpenAIGenerator(
        model="llama3-70b-8192",
        api_key=Secret.from_env_var("GROQ_API_KEY"),
        api_base_url="https://api.groq.com/openai/v1"
    )
)

rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")

def run_query(user_query):
    conversation_history.append(ChatMessage.from_user(user_query))

    response = rag_pipeline.run(
        {
            "text_embedder": {"text": user_query},
            "prompt_builder": {
                "query": user_query,
                "conversation_history": conversation_history
            }
        }
    )

    answer = response['llm']['replies'][0]

    conversation_history.append(ChatMessage.from_assistant(answer))

    if len(conversation_history) > 10:
        conversation_history[:] = conversation_history[-10:]

    return answer



d:\kerjo\python101\day-131-shop_recommendation_dibimbing-master\tf_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PromptBuilder has 3 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


In [ ]:
# Example usage
if __name__ == "__main__":
    user_query = "How can I cancel my order?"
    answer = run_query(user_query)
    print("Generated Answer:", answer)

    follow_up_query = "What are the refund policies for canceled orders?"
    answer = run_query(follow_up_query)
    print("Generated Answer:", answer)

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.19it/s]


Generated Answer: Based on the provided documents, to cancel your order, you can follow these steps:

1. Sign in to your account: Access the company's platform and log in using your credentials.
2. Navigate to your order history: Once logged in, find the section labeled "Order History" or "My Orders".
3. Locate your order: Look for the order with the specific order number and click on it.
4. Initiate the cancellation: Within the order details, find an option labeled "Cancel Order" and select it.
5. Confirm the cancellation: Follow any additional prompts or instructions to confirm the cancellation.

If you encounter any difficulties or have further questions, you can reach out to the customer support team during their available hours at the provided phone number or through the Live Chat feature on their website.


Batches: 100%|██████████| 1/1 [00:00<00:00, 12.49it/s]


Generated Answer: According to the provided documents, the refund policy for canceled orders is as follows:

* **Cancellation within Grace Period:** If you decide to cancel your purchase within the specified grace period, you may be eligible for a refund.
* **Event Cancellation:** If an event you have purchased tickets for is canceled and not rescheduled, you can typically obtain a refund.

Please note that specific vendors or service providers may have their own refund policies, so it's essential to review their terms and conditions for more detailed information.
